In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path, PurePath
import cupy as cp

In [ ]:
def xy_coords(coords_1d):
    x = coords_1d % 4096
    y = coords_1d // 4096
    return x, y


@cupy.fuse()
def dist_mat(x1, x2, y1, y2):
    return (x1 - x2)*(x1 - x2) + (y1-y2)*(y1-y2)


def dist_matrix(coords1, coords2):
    # Calculate distance matrix between 2 sets of coordinates. 
    # Inputs are 2 lists of spikes 1D-ravelled coordinates 
    
    # unravel to the 2D coordinate system of the CCD (4096 x 4096). This is faster than unravel() functions
    x1, y1 = xy_coords(coords1)
    x2, y2 = xy_coords(coords2)
    # Use broadcasting to get Euclidian distances. Seemed faster than using meshgrid functions.
    x1b = x1[:, np.newaxis]
    y1b = y1[:, np.newaxis]

    dist_matrix = dist_mat(x2, x1b, y2, y1b)#(x2 - x1b)**2 + (y2 - y1b)**2
    
    return dist_matrix


def is_near(coords1, coords2, distance):
    # Get a boolean array of same size as coords1 assigning True / False to its elements within 'distance' from coords2
    dmat = dist_matrix(coords1, coords2)
    near_mask = dmat <= distance
    is_near = near_mask.any(axis=1)
    return is_near

def is_near2(coords, distance):
    # Get a boolean array of same size as coords1 assigning True / False to its elements within 'distance' from coords2
    dmat = dist_matrix(coords, coords)
    near_mask = dmat <= distance
    is_near = near_mask.any(axis=1)
    return is_near

def is_near_w(coords_, distance, widx):
    # Get the pixels to keep within one file at a given wavelength
    isnear_bools = [is_near(coords_[widx], coords_[i], distance) for i in range(7)]
    isnear_bools[widx] = cupy.triu(isnear_bools[widx], k=1)
    mask_w_arr = cupy.vstack(isnear_bools)
    return mask_w_arr

def is_near_w2(coords, distance):
    # Get the pixels to keep within one file at a given wavelength
    isnear_bools = is_near2(coords, distance)
    mask_tri = cupy.triu(isnear_bools, k=1)
    return mask_tri

def create_mask_coincidentals_w_cupy(spikes_list, widx):
    
    cucoords_ = [cupy.asarray(spikes[0,:]) for spikes in spikes_list]
    mask_w_arr = is_near_w(cucoords_, 2, widx)
    select_pixels = mask_w_arr.any(axis=0)
    mask_w_arr2 = mask_w_arr[:, select_pixels]
    return select_pixels, mask_w_arr2

def create_mask_coincidentals_w_cupy2(spikes_coords, wslices):
    # Instead of looping through a list of 7 wavelengths, will merge the 7 arrays into 1. 
    mask_tri = is_near_w2(spikes_coords, 2)
    select_pixels = [mask_tri[:, wslice].any(axis=0) for wslice in wslices]
    return select_pixels

def extract_coincidentals_w_cupy(spikes_list, widx):
    
    select_pixels, mask_w_arr = create_mask_coincidentals_w_cupy(spikes_list, 0)
    select_pixels_h = select_pixels.get() # cupy.asnumpy(select_pixels)
    mask_w_arr_h = mask_w_arr.get() # cupy.asnumpy(mask_w_arr)
    return select_pixels_h, mask_w_arr_h

def extract_coincidentals_w_cupy2(spikes_coords, nspikes_per_wav):

    wslices = [slice(i*nspikes, (i+1)*nspikes) for i, nspikes in enumerate(nspikes_per_wav)]                               
    select_pixels = create_mask_coincidentals_w_cupy2(spikes_coords, wslices)
    select_pixels_h = [pixels.get() for pixels in select_pixels] # cupy.asnumpy(select_pixels)
    return select_pixels_h


## Load them in cpu RAM

In [ ]:
spikes_list = [cp.random.randint(1, high=4095*4095, size=[3, 8000]) for i in range(7)]

In [ ]:
a, b = extract_coincidentals_w_cupy(spikes_list, 0)

In [ ]:
%timeit a, b = extract_coincidentals_w_cupy(spikes_list, 0)

In [ ]:
g1 = cp.random.randint(1, high=4095*4095, size=[3, 8000])
g7 = cp.random.randint(1, high=4095*4095, size=8000 * 7, dtype=int)

In [ ]:
g7 = cp.random.randint(1, high=4095*4095, size=[8000, 8000], dtype=int)

In [ ]:
nspikes_per_wav = [8000]*7
wslices = [slice(i*nspikes, (i+1)*nspikes) for i, nspikes in enumerate(nspikes_per_wav)]

In [ ]:
isnear_bools = is_near2(g7, 2)

In [ ]:
isnear_bools = is_near2(g7, 2)
mask_w_arr = cupy.triu(isnear_bools, k=1)
select_pixels = [mask_w_arr[:, wslice].any(axis=0) for wslice in wslices]
cp.cuda.runtime.deviceSynchronize()

In [ ]:
%%timeit
isnear_bools = is_near2(g7, 2)
mask_w_arr = cupy.triu(isnear_bools, k=1)
cp.cuda.runtime.deviceSynchronize()

In [ ]:
%%timeit
isnear_bools = is_near2(g7, 2)
mask_w_arr = cupy.triu(isnear_bools, k=1)
select_pixels = [mask_w_arr[:, wslice].any(axis=0) for wslice in wslices]
cp.cuda.runtime.deviceSynchronize()

In [ ]:
select_pixels = create_mask_coincidentals_w_cupy2(g7, wslices)

In [ ]:
%%timeit 
select_pixels = create_mask_coincidentals_w_cupy2(g7, wslices)
cp.cuda.runtime.deviceSynchronize()

In [ ]:
select_pixels_cpu = extract_coincidentals_w_cupy2(g7, nspikes_per_wav)

In [ ]:
%%timeit
select_pixels_cpu = extract_coincidentals_w_cupy2(g7, nspikes_per_wav)

In [ ]:
select_pixels_cpu[0].shape

In [ ]:
%%timeit -n 10
g3 = cp.isin(g1, g2)
cp.cuda.runtime.deviceSynchronize()